In [83]:
# Импорт необходимых библиотек

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Задача 1

Расчет MAU

In [86]:
df = pd.read_csv('KC_case_data.csv', parse_dates=['date'])

In [87]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [88]:
df['dateym'] = df.date.dt.strftime('%Y-%m')

In [89]:
df.query('dateym=="2020-02"').agg({'device_id':'nunique'})[0]

75032

### Задача 2

Количество установок в январе

In [92]:
df.query('date.dt.month == 1').query('event == "app_install"').device_id.nunique()

80297

### Задача 3

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [95]:
purchase = df[df.event=='purchase'][['date','device_id','purchase_sum','utm_source']]
purchase = purchase.sort_values('date').drop_duplicates('device_id')

In [96]:
purchase.query('utm_source!="-"')\
        .groupby('utm_source',as_index=False).agg({'purchase_sum':'median'}).sort_values(by='purchase_sum',ascending=False)

,utm_source,purchase_sum
3,referal,395.5
2,instagram_ads,393.5
4,vk_ads,393.0
5,yandex-direct,392.5
1,google_ads,390.5
0,facebook_ads,389.0


### Задача 4

Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?

In [99]:
purch_facebook = df.query('utm_source=="facebook_ads"').agg({'purchase_sum':'sum'})[0]

purch_google = df.query('utm_source=="google_ads"').agg({'purchase_sum':'sum'})[0]

purch_instagram = df.query('utm_source=="instagram_ads"').agg({'purchase_sum':'sum'})[0]

purch_vk = df.query('utm_source=="vk_ads"').agg({'purchase_sum':'sum'})[0]

purch_yandex= df.query('utm_source=="yandex-direct"').agg({'purchase_sum':'sum'})[0]

In [100]:
print('ROMI for facebook:',round((purch_facebook-8590498)/8590498*100,2),'%') 
print('ROMI for google:',round((purch_google-10534878 )/10534878*100,2),'%')
print('ROMI for instagram:',round((purch_instagram-8561626)/8561626*100,2),'%')
print('ROMI for vk:',round((purch_vk-9553531)/9553531*100,2),'%')
print('ROMI for yandex:',round((purch_yandex-10491707)/10491707*100,2),'%')

ROMI for facebook: 42.6 %
ROMI for google: 22.15 %
ROMI for instagram: 69.91 %
ROMI for vk: 71.56 %
ROMI for yandex: 32.63 %
